# Classification: Good vs. Bad Road Conditions

Description: Classifying good and bad road conditions using accelerometer and gyroscope data without pothole labels.

In [106]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

### Get trip data from CSV

In [107]:
goodRoadFilePath = 'data/Good_Road_Bad_Road/good6_intervals.csv'
badRoadFilePath = 'data/Good_Road_Bad_Road/bad1_intervals.csv'
goodRoadDF = pd.read_csv(goodRoadFilePath)
badRoadDF = pd.read_csv(badRoadFilePath)
print goodRoadDF.shape
print badRoadDF.shape

(116, 15)
(52, 15)


In [108]:
a = goodRoadDF['condition']
b = badRoadDF['condition']
y = np.append(a,b)
print y
frames = [goodRoadDF, badRoadDF]
print y.shape
X = pd.concat(frames)
X = X[['avgSpeed', 'maxAccelX', 'maxAccelY', 'maxAccelZ', 'maxGyroX', 
       'maxGyroY', 'maxGyroZ', 'sdAccelX', 'sdAccelY', 'sdAccelZ',
       'sdGyroX','sdGyroY','sdGyroZ']]
print X[:3]

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.]
(168,)
   avgSpeed  maxAccelX  maxAccelY  maxAccelZ  maxGyroX  maxGyroY  maxGyroZ  \
0     0.916   0.195984  -0.928192   0.315994  0.001378  0.560718  0.005120   
1     2.700   0.242172  -0.922241   0.254822 -0.015193  0.691733 -0.101355   
2     3.277   0.335617  -0.870377   0.225876  0.017153  0.762203 -0.040848  

In [109]:
clf = svm.SVC(C=100)
#clf.fit(X, y)  
scores = cross_val_score(clf, X,y, cv=5)
np.mean(scores)

0.84588744588744602

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf.fit(X_train,y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [111]:
y_pred = clf.predict(X_test)
np.mean(y_pred==y_test)

0.9464285714285714

In [112]:
e = [1,2,3]
b = [1,2,3]
confusion_matrix(y_test, y_pred)

array([[37,  3],
       [ 0, 16]])

In [115]:
a = classification_report(y_test, y_pred)
print a

             precision    recall  f1-score   support

        0.0       1.00      0.93      0.96        40
        1.0       0.84      1.00      0.91        16

avg / total       0.95      0.95      0.95        56

